In [1]:
import os, re
from flask import Flask, render_template, request
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
import openai
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from PyPDF2 import PdfReader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.document_loaders import DirectoryLoader
from langchain.chains import LLMChain
import json

from langchain_community.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv



In [2]:
name = "shubham"
address = "pune"
claim_reason = "headache"
date = "2025-01-01"
hospital_name = "xyz"
medical_bill = r"D:\SHUBHAM\git_repo\GenAI-Project-InsuranceClaimAutomation\BillExtraction\Bills\Alzheimer.pdf"
description = "abc"

In [3]:
pdf_file = PdfReader(medical_bill)
bill = ""
for page in pdf_file.pages:
    bill = bill+page.extract_text()

print(bill)

APOLLO
HOSPIT ALS
Patient
Information:
-
Name:
Surbhit
-
Date
of
Birth:
01/01/1223
-
Address:
India
-
Phone
Number:1239874653
Service
Details:
Date
of
Service:
02/02/1233
Diagnosis:
Alzheimer’s
Details:
Continue
having
the
given
medicines
as
mentioned
in
the
prescription
Service
charges:
Doctor’s
fee
-
1500
Medicines
-
2000
Total
charge
-
3500
Membership
Discount
-
10%
Amount
payable
-
3150


In [4]:
from langchain.chat_models import ChatOllama
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import json


In [7]:
import yaml

# Load prompts
with open("../config/prompts.yaml", "r") as f:
    prompt_data = yaml.safe_load(f)

In [9]:
print(prompt_data["claim_result_template"])

Executive Summary
[Provide a Summary of the report.]

Introduction
[Write a paragraph about the aim of this report, and the state of the approval.]

Claim Details
[Provide details about the submitted claim]

Claim Description
[Write a short description about claim]

Document Verification
[Mentions which documents are submitted and if they are verified.] 

Document Summary
[Give a summary of everything here including the medical reports of the patient]  


In [5]:
system_prompt = """Act as an expert in extracting information from medical invoices. 
                    You are given with the invoice details of a patient. 
                    Go through the given document carefully and extract the 'disease' and the 'expense amount' from the data. 
                    Return only valid JSON in the exact format below. Do not include any explanation, comments, or extra text.
                    Format: {"disease": "", "expense": ""}  """

user_content = f"INVOICE DETAILS: {bill}"

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_content)
]

In [6]:
messages

[SystemMessage(content='Act as an expert in extracting information from medical invoices. \n                    You are given with the invoice details of a patient. \n                    Go through the given document carefully and extract the \'disease\' and the \'expense amount\' from the data. \n                    Return only valid JSON in the exact format below. Do not include any explanation, comments, or extra text.\n                    Format: {"disease": "", "expense": ""}  ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='INVOICE DETAILS: APOLLO\nHOSPIT ALS\nPatient\nInformation:\n-\nName:\nSurbhit\n-\nDate\nof\nBirth:\n01/01/1223\n-\nAddress:\nIndia\n-\nPhone\nNumber:1239874653\nService\nDetails:\nDate\nof\nService:\n02/02/1233\nDiagnosis:\nAlzheimer’s\nDetails:\nContinue\nhaving\nthe\ngiven\nmedicines\nas\nmentioned\nin\nthe\nprescription\nService\ncharges:\nDoctor’s\nfee\n-\n1500\nMedicines\n-\n2000\nTotal\ncharge\n-\n3500\nMembership\nDiscount\n-\n10%\

In [7]:
llm = ChatOllama(model="llama2", temperature=0.4)


C:\Users\S Karale\AppData\Local\Temp\ipykernel_14568\3604920378.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama2", temperature=0.4)


In [8]:
response = llm(messages)
print(response.content)

C:\Users\S Karale\AppData\Local\Temp\ipykernel_14568\3562080659.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


{"disease": "Alzheimer's", "expense": 3150}


In [9]:
json_data = re.search(r'\{.*\}', response.content).group()
json_data

'{"disease": "Alzheimer\'s", "expense": 3150}'

In [6]:
bill_info = json.loads(json_data)
bill_info

NameError: name 'json' is not defined

In [11]:
total_claim_amount = "5000"

In [12]:
if bill_info['expense'] != None and int(bill_info['expense']) < int(total_claim_amount) :
    claim_validation_message = "The amount mentioned for claiming is more than the billed amount. Claim Rejected."
    print("rejected")
    

rejected


In [13]:
patient_info = f"Name: {name}\nAddress: {address}\nClaim reason: {claim_reason}\nDate : {date}\nTotal claim amount: {total_claim_amount}\nDescription: {description}"
medical_bill_info = f"Medical Bill: {bill}"

In [14]:
print(patient_info)

Name: shubham
Address: pune
Claim reason: headache
Date : 2025-01-01
Total claim amount: 5000
Description: abc


In [15]:
# print(medical_bill_info)

In [16]:
doc_result_template = """            
            Executive Summary
            [Provide a Summary of the report.]

            Introduction
            [Write a paragraph about the aim of this report, and the state of the approval.]

            Claim Details
            [Provide details about the submitted claim]

            Claim Description
            [Write a short description about claim]

            Document Verification
            [Mentions which documents are submitted and if they are verified.] 

            Document Summary
            [Give a summary of everything here including the medical reports of the patient]"""

In [17]:
PROMPT = """You are an AI assistant for verifying health insurance claims. 
            You are given with the references for approving the claim and the patient details. 
            Analyse the given data and predict if the claim should be accepted or not. 
            Use the following guidelines for your analysis.

            1.Verify if the patient has provided all necessary information and all necessary documents
            and if you find any incomplete information or required documents are not 
            provided then set INFORMATION criteria as FALSE and REJECT the claim.
            if patient has provided all required documents then set INFORMATION criteria as TRUE. 

            2.If any disease mentioned in the medical bill of the patient is in the general exclusions list, 
            set EXCLUSION criteria as FALSE and REJECT the claim.

            Use this information to verify if the application is valid and to accept or reject the application.

            DOCUMENTS FOR CLAIM APPROVAL: {claim_approval_context}
            EXCLUSION LIST : {general_exclusion_context}
            PATIENT INFO : {patient_info}
            MEDICAL BILL : {medical_bill_info}

            Use the above information to verify if the application is valid and decide if the application has to be accepted or rejected keeping the guidelines into consideration. 

            Generate a detailed report about the claim and procedures you followed for accepting or rejecting the claim and the write the information you used for creating the report. 
            Create a report in the following format

            Write whether INFORMATION AND EXCLUSION are TRUE or FALSE 
            Reject the claim if any of them is FALSE.
            Write whether claim is accepted or not. If the claim has been accepted, the maximum amount which can be approved will be {max_amount}

            {doc_result_template}

            Please verify for any signs of fraud in the submitted claim if you find the documents required for accepting the claim for the medical treatment.
            """



In [18]:
general_exclusion_list = ["HIV/AIDS", 
                          "Parkinson's disease", 
                          "Alzheimer's disease",
                          "pregnancy", 
                          "substance abuse", 
                          "self-inflicted injuries", 
                          "sexually transmitted diseases(std)", 
                          "pre-existing conditions"]

claim_reason = bill_info["disease"]


In [19]:
print(patient_info,f"\n{claim_reason}")

Name: shubham
Address: pune
Claim reason: headache
Date : 2025-01-01
Total claim amount: 5000
Description: abc 
Alzheimer's


In [20]:
disease_in_list = any(claim_reason.lower() in disease.lower() for disease in general_exclusion_list)
disease_in_list

True

In [21]:
if disease_in_list:
    
    prompt_template = """You are an AI assistant for verifying health insurance claims. 
                         You are given with the references for approving the claim and the patient details. 
                         Analyse the given data and give a good rejection. 
                         You the following guidelines for your analysis.
                         PATIENT INFO : {patient_info}

                         {doc_result_template}
                        
                         CLAIM MUST BE REJECTED: Patient has {disease} which is present in the general exclusion list."""
else:
    prompt_template = PROMPT

validated_prompt = prompt_template

print(validated_prompt)


You are an AI assistant for verifying health insurance claims. 
                         You are given with the references for approving the claim and the patient details. 
                         Analyse the given data and give a good rejection. 
                         You the following guidelines for your analysis.
                         PATIENT INFO : {patient_info}

                         {doc_result_template}

                         CLAIM MUST BE REJECTED: Patient has {disease} which is present in the general exclusion list.


In [22]:
prompt_template1 = PromptTemplate(input_variables=["claim_approval_context","patient_info"],template=validated_prompt)


In [23]:
prompt_template1

PromptTemplate(input_variables=['disease', 'doc_result_template', 'patient_info'], input_types={}, partial_variables={}, template='You are an AI assistant for verifying health insurance claims. \n                         You are given with the references for approving the claim and the patient details. \n                         Analyse the given data and give a good rejection. \n                         You the following guidelines for your analysis.\n                         PATIENT INFO : {patient_info}\n\n                         {doc_result_template}\n\n                         CLAIM MUST BE REJECTED: Patient has {disease} which is present in the general exclusion list.')

In [24]:
llm = Ollama(model="llama2") 

C:\Users\S Karale\AppData\Local\Temp\ipykernel_14568\3646243349.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


In [25]:
llmchain = LLMChain(llm=llm, prompt= prompt_template1)
llmchain

C:\Users\S Karale\AppData\Local\Temp\ipykernel_14568\2751077865.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llmchain = LLMChain(llm=llm, prompt= prompt_template1)


LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['disease', 'doc_result_template', 'patient_info'], input_types={}, partial_variables={}, template='You are an AI assistant for verifying health insurance claims. \n                         You are given with the references for approving the claim and the patient details. \n                         Analyse the given data and give a good rejection. \n                         You the following guidelines for your analysis.\n                         PATIENT INFO : {patient_info}\n\n                         {doc_result_template}\n\n                         CLAIM MUST BE REJECTED: Patient has {disease} which is present in the general exclusion list.'), llm=Ollama(), output_parser=StrOutputParser(), llm_kwargs={})

NameError: name 'get_claim_approval_context' is not defined

In [27]:
## Load all document for insurance
document_path = r"D:\SHUBHAM\git_repo\GenAI-Project-InsuranceClaimAutomation\documents"
loader = DirectoryLoader(document_path, glob="**/*.pdf", show_progress=True, loader_cls=PyPDFLoader)
docs = loader.load()
docs

100%|██████████| 5/5 [00:13<00:00,  2.74s/it]


[Document(metadata={'producer': 'https://imagemagick.org\x00', 'creator': 'https://imagemagick.org\x00', 'creationdate': 'D:20240326124518', 'title': '1_MpSlUJoxPjb9jk6PG525vA\x00', 'author': 'https://imagemagick.org\x00', 'moddate': 'D:20240326124518', 'source': 'D:\\SHUBHAM\\git_repo\\GenAI-Project-InsuranceClaimAutomation\\documents\\med_bill1.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-03-26T05:16:07+00:00', 'source': 'D:\\SHUBHAM\\git_repo\\GenAI-Project-InsuranceClaimAutomation\\documents\\MembershipHandbook-22-23.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='General exclusions \n \n \n 19  \nThe Scheme shall not cover any treatment or expenses incurred directly or indirectly relating to: \n1. Pre-existing conditions – means illness or injury that commenced or presented sign(s) and \nsymptom(s), prior to the member’s coverage com

In [28]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter

In [29]:
type(docs)

list

In [30]:
chunks = text_splitter.split_documents(docs)

In [31]:
chunks

[Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-03-26T05:16:07+00:00', 'source': 'D:\\SHUBHAM\\git_repo\\GenAI-Project-InsuranceClaimAutomation\\documents\\MembershipHandbook-22-23.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='General exclusions \n \n \n 19  \nThe Scheme shall not cover any treatment or expenses incurred directly or indirectly relating to: \n1. Pre-existing conditions – means illness or injury that commenced or presented sign(s) and \nsymptom(s), prior to the member’s coverage commencement date. \n2. Treatment, medical service, medication or investigation which is not Medically Necessary. \n3. Any illness or bodily injury for which compensation is payable under any laws or regulations or \nany other insurance policy or any other sources except to the extent that such charges are not \nreimbursed by any such compensation, insurance policy or sources. \n4. Any charges for accommodation , nursing 

In [33]:
embeddings = OllamaEmbeddings()
docs_vector_store = FAISS.from_documents(chunks, embedding=embeddings)

In [50]:
docs_vector_store.save_local("faiss_vectore_store")
docs_vector_store

In [51]:
vector_store = FAISS.load_local("faiss_vectore_store", embeddings, allow_dangerous_deserialization=True)

In [ ]:
# retriever = docs_vector_store.as_retriever()

In [37]:
context = docs_vector_store.similarity_search("What are the documents required for claim approval?")
context

[Document(id='fcd20af0-23ef-4b16-9e9a-b3e9ff1ff796', metadata={'producer': 'Adobe PDF library 16.07', 'creator': 'Adobe Illustrator 26.3 (Windows)', 'creationdate': '2022-12-23T14:39:48+09:00', 'author': 'chanzend', 'moddate': '2022-12-23T14:39:48+08:00', 'title': 'New Tailor-made HB (A,B,D) 202301 - H+C or C only (C-BHN)_221219', 'source': 'D:\\SHUBHAM\\git_repo\\GenAI-Project-InsuranceClaimAutomation\\documents\\MembershipHandbook.pdf', 'total_pages': 53, 'page': 11, 'page_label': '12'}, page_content='Specialist * General Practitioner \nPhysiotherapist / Chiropractor / \nall Diagnostic Imaging & Laboratory Tests General Practitioner / Specialist \nPrescribed Western Medication General Practitioner / Specialist \n(issue “Prescription”) \nX-ray only and Laboratory Tests Registered Chinese Medicine Practitioner / \nChiropractor \nPsychological Counselling The attending Psychiatrist \n* Referral is not required for Dermatology, Family Medicine, Gynaecology, Ophthalmology, \nOrthopaedics,

In [42]:
context[0].page_content

'Specialist * General Practitioner \nPhysiotherapist / Chiropractor / \nall Diagnostic Imaging & Laboratory Tests General Practitioner / Specialist \nPrescribed Western Medication General Practitioner / Specialist \n(issue “Prescription”) \nX-ray only and Laboratory Tests Registered Chinese Medicine Practitioner / \nChiropractor \nPsychological Counselling The attending Psychiatrist \n* Referral is not required for Dermatology, Family Medicine, Gynaecology, Ophthalmology, \nOrthopaedics, Otolaryngology, Paediatric Surgery, Paediatrics and Psychiatry within HealthNet. \nReferral letter or prescription, if applicable, is valid for the same or related medical condition for \n6 months from the issue date.  Treatment for a new or unrelated medical condition will require \na new referral letter or prescription. \n \n \nIf you do not use BHN Card, please note: \n2 Tips on claiming for reimbursement of medical expenses:'

In [43]:
# context = docs_vector_store.similarity_search("What are the documents required for claim approval?")
claim_approval_context = ""
for x in context:
    claim_approval_context += x.page_content


In [45]:
print(claim_approval_context)

Specialist * General Practitioner 
Physiotherapist / Chiropractor / 
all Diagnostic Imaging & Laboratory Tests General Practitioner / Specialist 
Prescribed Western Medication General Practitioner / Specialist 
(issue “Prescription”) 
X-ray only and Laboratory Tests Registered Chinese Medicine Practitioner / 
Chiropractor 
Psychological Counselling The attending Psychiatrist 
* Referral is not required for Dermatology, Family Medicine, Gynaecology, Ophthalmology, 
Orthopaedics, Otolaryngology, Paediatric Surgery, Paediatrics and Psychiatry within HealthNet. 
Referral letter or prescription, if applicable, is valid for the same or related medical condition for 
6 months from the issue date.  Treatment for a new or unrelated medical condition will require 
a new referral letter or prescription. 
 
 
If you do not use BHN Card, please note: 
2 Tips on claiming for reimbursement of medical expenses:Claim settlement 
 Bupa will settle the expenses directly with the HealthNet Service Provid

In [46]:
context = docs_vector_store.similarity_search("Give a list of all general exclusions")
general_exclusion_context = ""
for x in context:
    general_exclusion_context += x.page_content

In [52]:
print(general_exclusion_context)

2. treatment by  a Non-HealthNet Specialist (only if referred by a Heal thNet General 
Practitioner when the necessary specialty is not available within Bupa HealthNet) 
Note: 
1. If pre-authorisation is required for emerg ency situations after Bupa’s office hours, 
subsequent authorisation can be done on the next working day after treatment.  
2. If you do not obtain pre-authorisation or pre-authorisation is not approved, all eligible 
medical expenses will be paid under Non-HealthNet Benefit, if any. 
HealthNet Registered Medical Practitioner fills in 
the Pre-authorisation Form and faxes it to Bupa. 
 
 
1 
 
 
Bupa sends the pre-authorisation confirmation to 
HealthNet Registered Medical Practitioner. 
 
 
2 
 
 You need to check and sign the pre-authorisation 
confirmation and keep the Member’s copy. 
 
 
 
3 
 
 
Pre-authorisation
proceduresindirect inguinal hernias.  
  
“Day Case” means a surgical procedure, Non-surgical Cancer Treatment and 
kidney dialysis (i f applicable und

In [54]:
output = llmchain.run({
    "claim_approval_context": claim_approval_context,
    "general_exclusion_context": general_exclusion_context,
    "patient_info": patient_info,
    "medical_bill_info": medical_bill_info,
    "max_amount": total_claim_amount,
    "disease": bill_info["disease"],
    "doc_result_template": doc_result_template
})

In [56]:
print(output)


Executive Summary:
This report outlines the reason for rejecting a health insurance claim submitted by Shubham, with the reference number XX-XXXX-XXXX. The claim was made on January 1st, 2025, for a total amount of Rs. 5000 due to a headache. However, upon further analysis, it has been found that Shubham has been diagnosed with Alzheimer's disease, which is present in the general exclusion list of the insurance policy. Therefore, the claim must be rejected.

Introduction:
The aim of this report is to provide a detailed analysis of the submitted claim and determine whether it should be approved or rejected based on the insurance policy terms and conditions. As per the information provided, Shubham has made a claim for Rs. 5000 due to a headache on January 1st, 2025. However, upon reviewing the patient's details, it has been found that Shubham has been diagnosed with Alzheimer's disease, which is present in the general exclusion list of the insurance policy.

Claim Details:
The claim su

In [58]:
print(re.sub(r'\n', '<br>', output))

<br>Executive Summary:<br>This report outlines the reason for rejecting a health insurance claim submitted by Shubham, with the reference number XX-XXXX-XXXX. The claim was made on January 1st, 2025, for a total amount of Rs. 5000 due to a headache. However, upon further analysis, it has been found that Shubham has been diagnosed with Alzheimer's disease, which is present in the general exclusion list of the insurance policy. Therefore, the claim must be rejected.<br><br>Introduction:<br>The aim of this report is to provide a detailed analysis of the submitted claim and determine whether it should be approved or rejected based on the insurance policy terms and conditions. As per the information provided, Shubham has made a claim for Rs. 5000 due to a headache on January 1st, 2025. However, upon reviewing the patient's details, it has been found that Shubham has been diagnosed with Alzheimer's disease, which is present in the general exclusion list of the insurance policy.<br><br>Claim 

In [ ]:
prompt = PromptTemplate(input_variables=["claim_approval_context", "general_exclusion_context", "patient_info","max_amount"], template=PROMPT)


In [ ]:
prompt 

PromptTemplate(input_variables=['claim_approval_context', 'general_exclusion_context', 'max_amount', 'medical_bill_info', 'patient_info'], input_types={}, partial_variables={}, template='You are an AI assistant for verifying health insurance claims. You are given with the references for approving the claim and the patient details. Analyse the given data and predict if the claim should be accepted or not. Use the following guidelines for your analysis.\n\n1.Verify if the patient has provided all necessary information and all necessary documents\nand if you find any incomplete information or required documents are not provided then set INFORMATION criteria as FALSE and REJECT the claim.\nif patient has provided all required documents then set INFORMATION criteria as TRUE. \n\n2. If any disease mentioned in the medical bill of the patient is in the general exclusions list, set EXCLUSION criteria as FALSE and REJECT the claim.\n\nUse this information to verify if the application is valid a